In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report

In [2]:
hacktest = pd.read_csv("hacktest.csv")
hacktrain = pd.read_csv("hacktrain.csv")


from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
hacktrain["class_encoded"] = le.fit_transform(hacktrain['class'])
hacktrain.columns

FileNotFoundError: [Errno 2] No such file or directory: 'hacktest.csv'

In [ ]:
ndvi_cols = hacktrain.columns[3:-1]
X=hacktrain[ndvi_cols]



In [ ]:
def remove_outliers(row):
  Q1 = row.quantile(0.25)
  Q3 = row.quantile(0.75)
  IQR = Q3 - Q1
  lower = Q1 - 1.5*IQR
  upper = Q3 + 1.5*IQR
  return row.where((row >= lower) & (row <= upper), np.nan)

In [ ]:
X=X.apply(remove_outliers, axis=1)
X.sample(10)

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
X.sample(10)

In [ ]:
sns.heatmap(X.isnull() , cbar =False , yticklabels=False , cmap='viridis')

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = pd.DataFrame(
    scaler.fit_transform(X),
    columns=X.columns,
    index=X.index
)
X.sample(10)

In [ ]:
X["mean"]=X.mean(axis=1)
X["median"]=X.median(axis=1)
X["var"]=X.var(axis=1)
X["min"]=X.min(axis=1)
X["max"]=X.max(axis=1)
X['range']=X.max(axis=1)-X.min(axis=1)
X.sample(4)


In [ ]:
Y = hacktrain['class_encoded']


In [ ]:
x_train , x_test , y_train , y_test = train_test_split(X,Y,test_size=0.2,random_state=42)



In [ ]:
from sklearn.model_selection import GridSearchCV


model = LogisticRegression(multi_class='multinomial',solver='newton-cg',max_iter=1000)
model.fit(x_train , y_train)
y_pred=model.predict(x_test)


print("accuracy is"  , accuracy_score(y_test , y_pred))
print("\n confusion matrix :\n", confusion_matrix(y_test , y_pred))
print("\n classification report \n",classification_report(y_test , y_pred))


In [ ]:
hacktest.head()

In [ ]:
hacktest_X = hacktest[ndvi_cols]

hacktest_X = hacktest_X.apply(remove_outliers, axis=1)
hacktest_X = pd.DataFrame(imputer.transform(hacktest_X), columns=hacktest_X.columns)



hacktest_X = pd.DataFrame(
    scaler.fit_transform(hacktest_X),
    columns=hacktest_X.columns,
    index=hacktest_X.index
)

hacktest_X["mean"]=hacktest_X.mean(axis=1)
hacktest_X["median"]=hacktest_X.median(axis=1)
hacktest_X["var"]=hacktest_X.var(axis=1)
hacktest_X["min"]=hacktest_X.min(axis=1)
hacktest_X["max"]=hacktest_X.max(axis=1)
hacktest_X['range']=hacktest_X.max(axis=1)-X.min(axis=1)
hacktest_X.sample(10)

In [ ]:
hacktest_y_pred = model.predict(hacktest_X)

decoded_pred = le.inverse_transform(hacktest_y_pred)

final_df = pd.DataFrame({'ID':hacktest['ID'].values , 'class':decoded_pred})

print(final_df.sample(8))

In [ ]:
final_df.to_csv("submission_6.csv",index=False)